t# Projeto 1 - Ciência dos Dados

Nome: Ariel Tamezgui Leventhal

Nome: Felipe Liberman Fuchs

Nome: Victoria Leal Garcia de Souza

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from emoji import UNICODE_EMOJI

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/home/ariel/naive-bayes-classifier


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'ps5.xlsx'

In [4]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Categoria,Unnamed: 2
0,@jamiltdavis @vicious696 i say the cutoff shou...,4,NaN
1,@rgt_85 @obe1plays when @rgt_85 tweets at you....,4,NaN
2,@one_gamergal6 @xboxstockalerts if you guys st...,3,NaN
3,@mbeasleysburner @wario64 they are literally l...,4,NaN
4,first couple followers to give us a private m...,4,NaN


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Categoria,gabas
0,"@cataferal hi tom, a couple things. in your f1...",NaN,1
1,@infamizmikep if the conversation is about gam...,NaN,2
2,@jte2409 @mcquack306 no free upgrade from ps4 ...,NaN,2
3,@abduplays_ just in time because i just got a ...,NaN,4
4,y’all gotta ps5?,NaN,4


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O produto escolhido foi o console Playstation 5 (PS5). Para fins de relevância, optamos seguir a seguinte lista: 
</br>
<li> Muito relevante: Tweets que possuem uma crítica (positiva ou negativa) sobre o produto que tenha fundamento em  alguma qualidade ou característica do console.
<li> Relevante: Tweets que possuem uma crítica (positiva ou negativa) sobre o produto sem especificar alguma qualidade ou característica do console.
<li> Pouco Relevante: Tweets que comentam sobre a falta de estoque do PS5 e/ou da dificuldade de comprar um console.
<li> Irrelevante: Tweets que não se encaixam em nenhuma das outras categorias, incluindo: memes, reclamações sobre algum jogo de PS5, promoções e sorteios para conseguir um PS5, entre outros.
</li>
</br>
Tweets que poderiam ser classificados como Relevante mas que contenham alguma proposta de solução a algum problema indicado nos Tweets foram classificados como Muito Relevante, mesmo não tendo comentário específico sobre alguma qualidade ou característica do console. 

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

* $P(Categoria|Texto) = \frac{P(Texto|Categoria) \cdot P(Categoria)}{P(Text)} = \frac{\sum_0^n P(Palavra|Categoria) \cdot P(Categoria)}{P(Text)}$

In [6]:
train.Categoria = train.Categoria.astype('category')
train.Categoria.cat.categories = ["Muito Relevante","Relevante", "Pouco Relevante", "Irrelevante"]
train.head()

,Treinamento,Categoria,Unnamed: 2
0,@jamiltdavis @vicious696 i say the cutoff shou...,Irrelevante,NaN
1,@rgt_85 @obe1plays when @rgt_85 tweets at you....,Irrelevante,NaN
2,@one_gamergal6 @xboxstockalerts if you guys st...,Pouco Relevante,NaN
3,@mbeasleysburner @wario64 they are literally l...,Irrelevante,NaN
4,first couple followers to give us a private m...,Irrelevante,NaN


In [7]:
special_letters = {",", "'", '"', ".", "!", "?", ";", ":", "[", "]","{", "}", "(", ")", "#", "/",  "\\", "https","+", "*", "-"}
stopwords = {'a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below',
            'between', 'both', 'but', 'by', "can't", 'cannot', 'could', "couldn't", 'did', "didn't", 'do', 'does', "doesn't", 'doing', "don't", 'down', 'during', 'each', 'few', 'for',
            'from', 'further', 'had', "hadn't", 'has', "hasn't", 'have', "haven't", 'having', 'he', "he'd", "he'll", "he's", 'her', 'here', "here's", 'hers', 'herself', 'him', 'himself',
            'his', 'how', "how's", 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', "isn't", 'it', "it's", 'its', 'itself', "let's", 'me', 'more', 'most', "mustn't", 'my',
            'myself', 'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'ought', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 'same', "shan't", 'she', "she'd",
            "she'll", "she's", 'should', "shouldn't", 'so', 'some', 'such', 'than', 'that', "that's", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', "there's", 'these',
            'they', "they'd", "they'll", "they're", "they've", 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 'very', 'was', "wasn't", 'we', "we'd", "we'll", "we're",
            "we've", 'were', "weren't", 'what', "what's", 'when', "when's", 'where', "where's", 'which', 'while', 'who', "who's", 'whom', 'why', "why's", 'with', "won't", 'would',
            "wouldn't", 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves'}

In [8]:
def is_not_user(word): # retorna se uma palavra é um nome de usuario 
    return not word[0] == "@"

def is_not_stopword(word): # retorna se uma palavra é uma stopword
    return not word in stopwords

def is_emoji(letter): # retorna se um caractere é um emoji
    return letter in UNICODE_EMOJI["en"]

def is_special(letter): # retorna se o caractere é um caractere especial
    return letter in special_letters

def clean_up(text):
    final_text= ""
    for letter in text:
        if is_emoji(letter):
            final_text += f" {letter} " # separa emojis
        elif is_special(letter):
            final_text += " " # filtra caracteres especiais com exceção de "_" para evitar atrapalhar o filtro de nome de usuarios
        else:
            final_text += letter

    return ' '.join(word.replace("_"," ") for word in final_text.split() if is_not_user(word) and is_not_stopword(word)).strip() # filtra usuarios, stopwords, caracteres especiais e "_"

In [9]:
train = train.drop("Unnamed: 2",axis=1)
train.head(1)

,Treinamento,Categoria
0,@jamiltdavis @vicious696 i say the cutoff shou...,Irrelevante


In [10]:
train.Categoria = train.Categoria.apply(lambda x: x.lower())

In [11]:
words_count = {cat: dict() for cat in train.Categoria.cat.categories}
print(words_count)
words = set()
for i, tweet_data in train.iterrows(): # itera pelas linhas do dataframe
    tweet = tweet_data.Treinamento # separa a coluna do tweet
    
    for word in clean_up(tweet).split(): # faz limpezas no texto
        words.add(word)
        cat_dict = words_count[tweet_data.Categoria]
        cat_dict[word] = cat_dict.get(word, 0)+1 # garante que a palavra existe no dicionario. caso exista o count é incrementado, caso contrário a palavra é adicionada com count=1
        
M = words_count['muito relevante']
R = words_count['relevante']
P = words_count['pouco relevante']
I = words_count['irrelevante']
I

{'muito relevante': {}, 'relevante': {}, 'pouco relevante': {}, 'irrelevante': {}}


{'say': 7,
 'cutoff': 2,
 'whenever': 2,
 'market': 1,
 'calms': 1,
 'anyone': 8,
 'wants': 2,
 'ps5': 336,
 'can': 29,
 'get': 57,
 'one': 28,
 'point': 4,
 'won': 3,
 't': 169,
 'wanting': 1,
 'play': 17,
 'game': 37,
 'settle': 1,
 'll': 7,
 'choice': 2,
 'consumer': 4,
 'tweets': 1,
 'things': 3,
 'real': 3,
 '🤣': 1,
 'personally': 3,
 'just': 41,
 'think': 13,
 'whole': 5,
 'upgrading': 1,
 'current': 2,
 'gen': 15,
 'confusing': 2,
 'used': 3,
 'able': 6,
 'plop': 1,
 '20': 2,
 'bucks': 3,
 'upgrade': 38,
 'hopefully': 3,
 'enough': 2,
 'people': 19,
 'know': 8,
 'deluxe': 8,
 'will': 27,
 'version': 42,
 'well': 6,
 'literally': 5,
 'like': 21,
 '2': 18,
 'mil': 2,
 'short': 3,
 'selling': 3,
 'much': 6,
 'first': 13,
 'couple': 5,
 'followers': 4,
 'give': 8,
 'us': 6,
 'private': 2,
 'message': 3,
 'sell': 3,
 'near': 2,
 'retail': 6,
 'price': 8,
 '❤': 3,
 '️': 10,
 '👇': 2,
 '🏻': 1,
 'https': 135,
 'co': 135,
 'kieqvoqtzv': 1,
 'missed': 4,
 'drop': 4,
 'shoot': 2,
 'dm': 5,


In [12]:
total = len(words)         # total de palavras no banco de dados
totalM = (sum(M.values())) # total de palavras na categoria Muito Relevante
totalR = (sum(R.values())) # total de palavras na categoria Relevante
totalP = (sum(P.values())) # total de palavras na categoria Pouco Relevante
totalI = (sum(I.values())) # total de palavras na categoria Irrelevante

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [13]:
from pandas.api.types import CategoricalDtype
l = [] # lista de classificações, mesma ordem de test

for i, tweet_data in test.iterrows():  # itera pelas linhas do dataframe
    # valores para a probabiliade do tweet ser determinada relevancia, começa em 1 (elemento neutro da multiplicação)
    i = 1
    p = 1
    r = 1
    m = 1
    tweet = tweet_data.Teste # separa a coluna do tweet
    
    # faz limpezas no texto
    for word in clean_up(tweet).split(): 
        
        # calcula as probabilidades de cada relevancia fazendo suavizavao de laplace
        r *= (R.get(word, 0) + 1) / (totalR + total) 
        p *= (P.get(word, 0) + 1) / (totalP + total) 
        i *= (I.get(word, 0) + 1) / (totalI + total) 
        m *= (M.get(word, 0) + 1) / (totalM + total)
        
    result = {"Muito Relevante": m,"Relevante": r,"Pouco Relevante": p,"Irrelevante": i} # dicionário para comparar dentro do MAX abaixo
    
    l.append(max(result, key = result.get)) # compara os valores de r, p, i, e adiciona à lista L 

test.Categoria = pd.Categorical(values=l,categories=["Muito Relevante","Relevante", "Pouco Relevante", "Irrelevante"],ordered=True)

In [14]:
test.gabas = test.gabas.astype("category")
test.gabas.cat.categories = ["Muito Relevante","Relevante", "Pouco Relevante", "Irrelevante"]

In [15]:
test.head(80)

,Teste,Categoria,gabas
0,"@cataferal hi tom, a couple things. in your f1...",Muito Relevante,Muito Relevante
1,@infamizmikep if the conversation is about gam...,Relevante,Relevante
2,@jte2409 @mcquack306 no free upgrade from ps4 ...,Irrelevante,Relevante
3,@abduplays_ just in time because i just got a ...,Irrelevante,Irrelevante
4,y’all gotta ps5?,Relevante,Irrelevante
...,...,...,...
75,what's #best today on https://t.co/cxcprnejia ...,Irrelevante,Irrelevante
76,@destinlegarie i mean at this point that shoul...,Irrelevante,Irrelevante
77,i literally love sony wait as someone who’s ne...,Irrelevante,Irrelevante
78,@gloriouskev @thicc_stick_boi it's still one p...,Irrelevante,Relevante


In [16]:
pd.crosstab(test.Categoria,test.gabas)

gabas,Muito Relevante,Relevante,Pouco Relevante,Irrelevante
Categoria,,,,
Muito Relevante,5,2,1,22
Relevante,8,18,3,52
Pouco Relevante,0,0,27,4
Irrelevante,1,16,5,86


In [17]:
pd.crosstab(test.Categoria,test.gabas,normalize="index").round(4)*100

gabas,Muito Relevante,Relevante,Pouco Relevante,Irrelevante
Categoria,,,,
Muito Relevante,16.67,6.67,3.33,73.33
Relevante,9.88,22.22,3.70,64.20
Pouco Relevante,0.00,0.00,87.10,12.90
Irrelevante,0.93,14.81,4.63,79.63


In [18]:
pd.crosstab(test.Categoria,test.gabas,normalize="columns").round(4)*100

gabas,Muito Relevante,Relevante,Pouco Relevante,Irrelevante
Categoria,,,,
Muito Relevante,35.71,5.56,2.78,13.41
Relevante,57.14,50.00,8.33,31.71
Pouco Relevante,0.00,0.00,75.00,2.44
Irrelevante,7.14,44.44,13.89,52.44


In [19]:
pd.crosstab(test.Categoria,test.gabas,normalize=True).round(4)*100

gabas,Muito Relevante,Relevante,Pouco Relevante,Irrelevante
Categoria,,,,
Muito Relevante,2.0,0.8,0.4,8.8
Relevante,3.2,7.2,1.2,20.8
Pouco Relevante,0.0,0.0,10.8,1.6
Irrelevante,0.4,6.4,2.0,34.4


In [20]:
print(totalM)
print(totalR)
print(totalP)
print(totalI)

592
1870
1827
5328


___
### Concluindo

In [21]:
train.Categoria.value_counts()

irrelevante        312
relevante           88
pouco relevante     74
muito relevante     26
Name: Categoria, dtype: int64

In [22]:
test.gabas.value_counts()

Irrelevante        164
Relevante           36
Pouco Relevante     36
Muito Relevante     14
Name: gabas, dtype: int64

In [23]:
test.Categoria.value_counts()

Irrelevante        108
Relevante           81
Pouco Relevante     31
Muito Relevante     30
Name: Categoria, dtype: int64

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

In [24]:
df1 = pd.read_excel(filename)
df2 = pd.read_excel(filename, sheet_name = 'Teste')
df1 = df1.drop("Unnamed: 2",axis=1)
df2 = df2.drop("Categoria",axis=1)
df1 = df1.rename(columns={"Treinamento":"Tweets"})
df2 = df2.rename(columns={"Teste":"Tweets","gabas":"Categoria"})

In [25]:
tweets = pd.concat([df1,df2],ignore_index=True)
tweets.head(3)

,Tweets,Categoria
0,@jamiltdavis @vicious696 i say the cutoff shou...,4
1,@rgt_85 @obe1plays when @rgt_85 tweets at you....,4
2,@one_gamergal6 @xboxstockalerts if you guys st...,3


In [26]:
tweets = tweets.sample(frac=1)
tweets.head(3)

,Tweets,Categoria
259,@scythebiscuit @curryspcworld i have a ps5 nev...,4
171,@lbabinz more ps5 stock soon?,3
699,@agnaktorisbest look at it yourself bruv. it’s...,4


___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**